# Batch Processing for IntroDS Milestone 2
# 
This notebook automates the pipeline for all papers found in `data_raw`.
It performs:
1.  **Multi-file Gathering:** Finds root tex and flattens structure [cite: 11-14].
2.  **Hierarchy Construction:** Parses LaTeX into a tree structure [cite: 15-20].
3.  **Standardization:** Cleans content and splits sentences [cite: 27-30].
4.  **Content Deduplication:** Merges content across versions [cite: 36-37].
5.  **Output Generation:** Saves `hierarchy.json` in the required folder structure [cite: 89-96].


In [2]:

# %%
import sys
import os
import json
import shutil
from tqdm import tqdm # Thư viện tạo thanh progress bar (nếu chưa có: pip install tqdm)

# --- SETUP PATHS ---
# Adjust project_root if necessary
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '23127011'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import các module đã viết
from src.parser import (
    find_root_tex_file, 
    LatexFlattener, 
    LatexStructureBuilder, 
    LatexContentProcessor
)
from src.processing import ContentDeduplicator
# , ReferenceExtractor 
# Lưu ý: Import ReferenceExtractor nếu bạn muốn xuất cả refs.bib


In [3]:

# Define Data Paths
DATA_RAW_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'data_raw'))
DATA_OUTPUT_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'data_output'))

print(f"📂 Input: {DATA_RAW_PATH}")
print(f"📂 Output: {DATA_OUTPUT_PATH}")


📂 Input: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\data_raw
📂 Output: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\data_output


## Helper Functions

Hàm hỗ trợ để lấy danh sách thư mục con và sắp xếp version.

In [4]:
def get_subdirs(path):
    """Returns a list of immediate subdirectories in a path."""
    if not os.path.exists(path):
        return []
    return [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

def sort_versions(version_list):
    """Sorts versions logically: v1, v2, v10... instead of v1, v10, v2"""
    try:
        return sorted(version_list, key=lambda x: int(x.split('v')[-1]) if 'v' in x else 0)
    except:
        return sorted(version_list)


## Main Processing Pipeline

Vòng lặp chính duyệt qua từng bài báo và từng version.


In [5]:
# Tạo folder output gốc nếu chưa có
if not os.path.exists(DATA_OUTPUT_PATH):
    os.makedirs(DATA_OUTPUT_PATH)

# Lấy danh sách các bài báo (Paper IDs)
papers = get_subdirs(DATA_RAW_PATH)
print(f"Found {len(papers)} papers to process: {papers}")


Found 11 papers to process: ['2403-00530', '2403-00531', '2403-00532', '2403-00533', '2403-00534', '2403-00535', '2403-00536', '2403-00537', '2403-00538', '2403-00539', '2403-00540']


In [6]:

# Duyệt qua từng bài báo
for paper_id in tqdm(papers, desc="Processing Papers"):
    print(f"\n📘 Processing Paper: {paper_id}")
    
    # 1. Setup Output Folder cho bài báo này
    # Cấu trúc: data_output/<paper_id>/
    paper_output_dir = os.path.join(DATA_OUTPUT_PATH, paper_id)
    if not os.path.exists(paper_output_dir):
        os.makedirs(paper_output_dir)
        
    # 2. Setup Deduplicator (Mỗi bài báo dùng 1 dedup riêng để gộp các version của nó)
    deduplicator = ContentDeduplicator()
    
    # 3. Tìm các version trong folder tex/
    tex_source_dir = os.path.join(DATA_RAW_PATH, paper_id, 'tex')
    if not os.path.exists(tex_source_dir):
        print(f"   ⚠️ 'tex' folder not found for {paper_id}. Skipping.")
        continue
        
    versions = sort_versions(get_subdirs(tex_source_dir))
    
    if not versions:
        print(f"   ⚠️ No versions found in {tex_source_dir}. Skipping.")
        continue

    # --- PROCESS TỪNG VERSION ---
    for ver in versions:
        ver_path = os.path.join(tex_source_dir, ver)
        print(f"   👉 Version: {ver}")
        
        try:
            # A. Tìm Root File [cite: 13]
            root_file = find_root_tex_file(ver_path)
            if not root_file:
                print(f"      ❌ Root file not found in {ver}. Skipping.")
                continue
            
            # B. Flatten LaTeX [cite: 11-12]
            # Gộp tất cả file \input, \include thành 1 chuỗi
            flattener = LatexFlattener(root_file, paper_id=paper_id, version=ver)
            flat_result = flattener.flatten()
            flat_content_str = flat_result['content'] # Lấy string nội dung
            
            # C. Build Structure Tree [cite: 15-20]
            # Parse thành cây section/chapter/subsection
            builder = LatexStructureBuilder(flat_content_str, paper_id, ver)
            root_node = builder.build_coarse_tree() # Hoặc .parse() tùy tên hàm bạn đang dùng
            
            # D. Process Content (Clean & Split Sentences) [cite: 23-24, 28-30]
            # Tách câu, xử lý công thức toán, hình ảnh
            processor = LatexContentProcessor(paper_id, ver)
            processor.process_tree(root_node)
            
            # E. Add to Deduplicator [cite: 36-37]
            # Gộp vào kho chung, xử lý trùng lặp nội dung
            deduplicator.process_version(ver, root_node)
            
            # (Optional) F. Extract References [cite: 31]
            # Nếu bạn đã tích hợp ReferenceExtractor, hãy gọi ở đây để lấy refs cho file refs.bib
            # ref_extractor = ReferenceExtractor()
            # refs = ref_extractor.extract_from_content(flat_content_str)
            # ... xử lý dedup reference ...

        except Exception as e:
            print(f"      ❌ Error processing {ver}: {str(e)}")
            # import traceback
            # traceback.print_exc()

    # --- EXPORT KẾT QUẢ CHO BÀI BÁO NÀY ---
    
    # 1. Export hierarchy.json [cite: 106-133]
    final_json = deduplicator.get_final_json()
    hierarchy_path = os.path.join(paper_output_dir, 'hierarchy.json')
    
    with open(hierarchy_path, 'w', encoding='utf-8') as f:
        json.dump(final_json, f, indent=2, ensure_ascii=False)
    
    print(f"   ✅ Saved hierarchy.json ({len(final_json['elements'])} unique elements)")
    
    # 2. Copy metadata.json và references.json (nếu có trong raw) [cite: 98-104]
    # Vì code parser không tạo metadata, ta copy từ nguồn (thường scraper đã lấy về)
    for meta_file in ['metadata.json', 'references.json']:
        src_meta = os.path.join(DATA_RAW_PATH, paper_id, meta_file)
        dst_meta = os.path.join(paper_output_dir, meta_file)
        if os.path.exists(src_meta):
            shutil.copy2(src_meta, dst_meta)
            print(f"   ✅ Copied {meta_file}")
        else:
            print(f"   ⚠️ Missing {meta_file} in source.")

print("\n🎉 ALL DONE! Check 'data_output' folder.")

Processing Papers:   0%|          | 0/11 [00:00<?, ?it/s]


📘 Processing Paper: 2403-00530
   👉 Version: 2403-00530v1
📝 Khởi tạo LatexFlattener cho Paper: 2403-00530, Version: 2403-00530v1
🔍 Xử lý Preamble để trích xuất Title, Authors, Abstract...
🔄 Processing hierarchy for version: 1 (from 2403-00530v1)
   👉 Version: 2403-00530v2
📝 Khởi tạo LatexFlattener cho Paper: 2403-00530, Version: 2403-00530v2
🔍 Xử lý Preamble để trích xuất Title, Authors, Abstract...


Processing Papers:   9%|▉         | 1/11 [00:00<00:01,  5.55it/s]

🔄 Processing hierarchy for version: 2 (from 2403-00530v2)
   ✅ Saved hierarchy.json (360 unique elements)
   ✅ Copied metadata.json
   ✅ Copied references.json

📘 Processing Paper: 2403-00531
   👉 Version: 2403-00531v1
📝 Khởi tạo LatexFlattener cho Paper: 2403-00531, Version: 2403-00531v1
🔍 Xử lý Preamble để trích xuất Title, Authors, Abstract...
🔄 Processing hierarchy for version: 1 (from 2403-00531v1)


Processing Papers:  27%|██▋       | 3/11 [00:00<00:00,  8.97it/s]

...

🔄 Processing hierarchy for version: 1 (from 2403-00535v1)
   ✅ Saved hierarchy.json (892 unique elements)
   ✅ Copied metadata.json
   ✅ Copied references.json

📘 Processing Paper: 2403-00536
   👉 Version: 2403-00536v1


Processing Papers:  82%|████████▏ | 9/11 [00:01<00:00,  7.53it/s]

📝 Khởi tạo LatexFlattener cho Paper: 2403-00536, Version: 2403-00536v1
🔍 Xử lý Preamble để trích xuất Title, Authors, Abstract...
🔄 Processing hierarchy for version: 1 (from 2403-00536v1)
   ✅ Saved hierarchy.json (1307 unique elements)
   ✅ Copied metadata.json
   ✅ Copied references.json

📘 Processing Paper: 2403-00537
   👉 Version: 2403-00537v1
📝 Khởi tạo LatexFlattener cho Paper: 2403-00537, Version: 2403-00537v1
🔍 Xử lý Preamble để trích xuất Title, Authors, Abstract...
🔄 Processing hierarchy for version: 1 (from 2403-00537v1)
   👉 Version: 2403-00537v2
📝 Khởi tạo LatexFlattener cho Paper: 2403-00537, Version: 2403-00537v2
🔍 Xử lý Preamble để trích xuất Title, Authors, Abstract...
🔄 Processing hierarchy for version: 2 (from 2403-00537v2)
   ✅ Saved hierarchy.json (198 unique elements)
   ✅ Copied metadata.json
   ✅ Copied references.json

📘 Processing Paper: 2403-00538
   👉 Version: 2403-00538v1
📝 Khởi tạo LatexFlattener cho Paper: 2403-00538, Version: 2403-00538v1
🔍 Xử lý Preambl

Processing Papers: 100%|██████████| 11/11 [00:01<00:00,  7.91it/s]

📝 Khởi tạo LatexFlattener cho Paper: 2403-00540, Version: 2403-00540v1
🔍 Xử lý Preamble để trích xuất Title, Authors, Abstract...
🔄 Processing hierarchy for version: 1 (from 2403-00540v1)
   👉 Version: 2403-00540v2
📝 Khởi tạo LatexFlattener cho Paper: 2403-00540, Version: 2403-00540v2
🔍 Xử lý Preamble để trích xuất Title, Authors, Abstract...
🔄 Processing hierarchy for version: 2 (from 2403-00540v2)
   👉 Version: 2403-00540v3
📝 Khởi tạo LatexFlattener cho Paper: 2403-00540, Version: 2403-00540v3
🔍 Xử lý Preamble để trích xuất Title, Authors, Abstract...
🔄 Processing hierarchy for version: 3 (from 2403-00540v3)
   ✅ Saved hierarchy.json (523 unique elements)
   ✅ Copied metadata.json
   ✅ Copied references.json

🎉 ALL DONE! Check 'data_output' folder.
